# Getting Data

First, we want to grab some graphs and subject covariates from a web-accessible url.  We've given this to you on google drive rather than having you set up aws s3 credentials in the interest of saving time. The original data is hosted at m2g.io

Below, you will be getting the following dataset:

| Property | Value |
|:--------:|:-----:|
| Dataset  | SWU4  |
| N-Subjects  | 454   |
| Scans-per-subjects | 2 |
| Atlases | Desikan, CPAC200 |
| Desikan Nodes | 70 |
| CPAC200 Nodes | 200 |

The covariates you have are: `SUBID, SESSION, AGE_AT_SCAN_1, SEX, RESTING_STATE_INSTRUCTION, TIME_OF_DAY, SEASON, SATIETY, LMP`. There are other columns in the `.csv` file (downloaded in the next step) but they are populated with a `#` meaning that the value was not recorded.

There are several other atlases available - you can change which one you use 
Running the cell below will get you the data. **Please note, you only have to run these two cells once!!!**

## Loading Graphs + Covariates
Run the following cells of code to load the graphs into your computer, as well as the covariates.

In [16]:
!pip install networkx==1.9 #networkx broke backwards compatibility with these graph files
import numpy as np
import networkx as nx
import scipy as sp
import matplotlib.pyplot as plt
import os
import csv


from collections import OrderedDict

In [17]:
# Initializing dataset names
dataset_names = 'SWU4'

basepath = 'data'

# change which atlas you use, here!

atlas = 'desikan' # 'desikan' # or 'CPAC200', or 'Talairach'
dir_names = basepath + '/' + dataset_names + '/' + atlas
#basepath = "/"
#dir_names = basepath
print(dir_names)
fs = OrderedDict()
fs[dataset_names] = [root + "/" + fl for root, dirs, files in os.walk(dir_names)
                     for fl in files if fl.endswith(".gpickle")]

ps = "data/SWU4/SWU4.csv"

print("Datasets: " + ", ".join([fkey + " (" + str(len(fs[fkey])) + ")"
                                for fkey in fs]))
print("Total Subjects: %d" % (sum([len(fs[key]) for key in fs])))

data/SWU4/Talairach
Datasets: SWU4 (454)
Total Subjects: 454


In [18]:
def loadGraphs(filenames, verb=False):
    """
    Given a list of files, returns a dictionary of graphs

    Required parameters:
        filenames:
            - List of filenames for graphs
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Initializes empty dictionary
    gstruct = OrderedDict()
    for idx, files in enumerate(filenames):
        if verb:
            print("Loading: " + files)
        #  Adds graphs to dictionary with key being filename
        fname = os.path.basename(files)
        gstruct[fname] = nx.read_gpickle(files)
    return gstruct

def constructGraphDict(names, fs, verb=False):
    """
    Given a set of files and a directory to put things, loads graphs.

    Required parameters:
        names:
            - List of names of the datasets
        fs:
            - Dictionary of lists of files in each dataset
    Optional parameters:
        verb:
            - Toggles verbose output statements
    """
    #  Loads graphs into memory for all datasets
    graphs = OrderedDict()
    if verb:
        print("Loading Dataset: " + names)
    # The key for the dictionary of graphs is the dataset name
    graphs[names] = loadGraphs(fs[names], verb=verb)
    return graphs

In [19]:
graphs = constructGraphDict(dataset_names, fs, verb=False)

UnicodeDecodeError: 'ascii' codec can't decode byte 0x80 in position 0: ordinal not in range(128)

In [ ]:
import csv
# This gets age and sex, respecitvely.
tmp = csv.reader(open(ps,newline='')) # this is the whole phenotype file
pheno = OrderedDict()
triple = [[t[0].strip(), t[2], int(t[3] == '2')] for t in tmp
          if t[3] != '#' and t[2] != '#'][1:]  # female=1->0, male=2->1

for idx, trip in enumerate(triple):
    pheno[trip[0]] = trip[1:]

In [ ]:
k = list(graphs['SWU4'].keys())
g = list(graphs['SWU4'].values())
k = list(key[6:11] for key in k)
k = k[0::2]
g1 = g[0::2]
g2 = g[1::2]
d = dict(zip(k,g1))

#Create vectors of labels
age = list()
sex = list()

for key in k:
    sex.append(pheno[key][1])
    age.append(pheno[key][0])
      
        
# prints number of discrepencies between data and processed data
c = 0
for i in range(len(g1)):
    if sex[i] != pheno[k[i]][1]:
        c += 1
print (c)
        
# should use g1 for now - g2 is the retest data for each subject        

## ASSIGNMENT:  
(Code above used to get data in the correct format.  Below is a simple example test string with kind of silly features)

In [ ]:
#Combine features, separate training and test data

X = []
for i in range(len(g1)):
    featvec = []
#     print (k[i])
    
    
    matrix = nx.to_numpy_matrix(g1[i], nodelist=sorted(g1[i].nodes())) #this is how you go to a matrix
    
    """cc = nx.closeness_centrality(g1[i])
    for j in list(cc.values()):
        featvec.append(float(j))"""
    
    bc = nx.betweenness_centrality(g1[i])
    for j in list(bc.values()):
        featvec.append(float(j))
        
    # Add each edge
    for edge in np.ravel(matrix):
        featvec.append(edge)
    
    #featvec.extend(np.ravel(np.log(np.sum(matrix,0) + 1)))
    featvec.append(nx.average_clustering(g1[i]))
    #featvec.append(np.mean(matrix))"""
    
    np.shape(featvec)
    X.append(featvec)


In [ ]:
X_train = X[0:100]
Y_train = sex[0:100]

X_test = X[100:200]
Y_test = sex[100:200]

from sklearn.ensemble import RandomForestClassifier

print ("Random Forest: ")
accuracy = []

for ii in range(50): #performance will change over time
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, Y_train)
    acc = (clf.predict(X_test) == Y_test)
    #print(acc)
    accval = (float(np.sum(acc))/float(len(Y_test)))
    accuracy.append(accval)
#     print('Accuracy:',accval)
    
print('Overall Accuracy:',str(np.mean(accuracy)))

In [ ]:
from sklearn.naive_bayes import GaussianNB

nbclf = GaussianNB()
nbclf.fit(X_train, Y_train)
acc = (clf.predict(X_test) == Y_test)
accval = (float(np.sum(acc))/float(len(Y_test)))
accuracy.append(accval)
print('Naive Bayes Accuracy:',accval)


In [ ]:
# plot a graph
import matplotlib.pyplot as plt
%matplotlib inline

females = []
males = []
allSubjects = []

for i in range(len(g1)):
    # Females are 0, males are 1
    matrix = nx.to_numpy_matrix(g1[i], nodelist=sorted(g1[i].nodes()))
    allSubjects.append(matrix)
    if (sex[i] == 0):
        females.append(matrix)
    else:
        males.append(matrix)

# Take the average along the first axis only
matrixA = np.mean(allSubjects, axis = 0);
plt.imshow(np.log10(matrixA+1))
plt.colorbar()
plt.title('Mean Connectome')
plt.show()
        
# Take the average along the first axis only
matrixM = np.mean(males, axis = 0);
plt.imshow(np.log10(matrixM+1))
plt.colorbar()
plt.title('Mean Male Connectome')
plt.show()

matrixF = np.mean(females, axis = 0);
plt.imshow(np.log10(matrixF + 1))
plt.colorbar()
plt.title('Mean Female Connectome')
plt.show()

In [ ]:
# Test retest
# We simply use the total difference between each matrix
# Does work!!!
for i in range(len(g1)):
    m1 = nx.to_numpy_matrix(g1[i], nodelist=sorted(g1[i].nodes()))
    m2 = nx.to_numpy_matrix(g2[i], nodelist=sorted(g2[i].nodes()))    
    self = np.sum(np.log(np.absolute(np.diff([m1, m2], 0)) + 1))
    
    # Average of other diffs
    other = 0
    for j in range(len(g1)):
        if (i == j): continue
        m3 = nx.to_numpy_matrix(g2[j], nodelist=sorted(g2[j].nodes()))
        
        other += np.sum(np.log(np.absolute(np.diff([m1, m3], 0)) + 1))
    other /= float(len(g1)-1)
    print(self, other)
